In [ ]:
import sys

sys.path.append("..")
# this is what you need for python script
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ParTIpy.arch import AA
from ParTIpy.generate_test_data import simulate
from ParTIpy.const import OPTIM_ALGS

N_SAMPLES = 20_000
N_ARCHETYPES = 8
N_DIMENSIONS = 20

X, A, Z = simulate(n_samples=N_SAMPLES, 
                   n_archetypes=N_ARCHETYPES, 
                   n_dimensions=N_DIMENSIONS, 
                   noise_std=0.0)

In [ ]:
df_list = []
for optim_alg in OPTIM_ALGS:
    AA_object = AA(n_archetypes=N_ARCHETYPES, 
                   weight=None,
                   optim=optim_alg, 
                   init="random",
                   max_iter=100).fit(X)
    df_list.append(pd.DataFrame({
        "rss": AA_object.RSS_trace.copy(),
        "iteration": np.arange(len(AA_object.RSS_trace)),
        "algorithm": optim_alg
    }))
plot_df = pd.concat(df_list, axis=0)
plot_df

In [ ]:
plt.style.use("dark_background")
sns.lineplot(data=plot_df, x="iteration", y="rss", hue="algorithm")
plt.xlabel("Iteration")
plt.ylabel("Residual Sum of Squares (RSS)")
plt.show()